In [2]:
# !apt-get update
# !pip install selenium
# !pip install python_dotenv
# !pip install opencv-python-headless
# !pip install Pillow

import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
import re
import numpy as np
import pandas as pd
import uuid
from PIL import Image,ImageEnhance
import pytesseract
from io import BytesIO
import time  
from pytesseract import image_to_string
import cv2

In [3]:
class DriverManager:
    def __init__(self):
        # Initialize Edge options
        self.options = EdgeOptions()
        self.options.add_argument('--start-maximized')  
        service = EdgeService("/home/keshu/Documents/Encumbrance_Automate/msedgedriver")  
        self.driver = webdriver.Edge(service=service, options=self.options)
    
    def get_driver(self):
        return self.driver

    def close_driver(self):
        self.driver.quit()

In [4]:
class SiteNavigator:
    def __init__(self, driver):
        self.driver = driver
        
    def translate_page(self):
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "body > form:nth-child(9) > div:nth-child(4) > a:nth-child(11)"))
            )
            self.driver.find_element(By.CSS_SELECTOR,"body > form:nth-child(9) > div:nth-child(4) > a:nth-child(11)").click()
            print("Page translated successfully.")
        except Exception as e:
            print(f"Translation feature not found or failed. Error: {str(e)}")
        

    def search_records(self):
        self.translate_page()
        self.driver.implicitly_wait(10);
        self.driver.find_element(By.CSS_SELECTOR,"li[onclick='loadECData()']").click()
        

In [5]:
class ZoneHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def select_zone(self):
        #implemented for only "Chengalpattu"
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_Zone").click()
        WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='15']"))
        )
        self.driver.find_element(By.CSS_SELECTOR,"option[value='15']").click()
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_Zone").click()

In [6]:
class DistrictHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def select_district(self):
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_District").click()
        WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='20004']"))
        )
        self.driver.find_element(By.CSS_SELECTOR,"option[value='20004']").click()
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_District").click()

In [7]:
class RegistrarHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def select_registrar(self):
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_SroName").click()
        WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='20088:1']"))
        )
        self.driver.find_element(By.CSS_SELECTOR,"option[value='20088:1']").click()
        self.driver.find_element(By.CSS_SELECTOR,"#cmb_SroName").click()

In [8]:
class DateHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def populate_dates(self):
        start_date=self.driver.find_element(By.ID,"txt_PeriodStartDt")
        start_date.clear()
        self.driver.implicitly_wait(10);
        self.driver.execute_script("arguments[0].value = '01-Jan-2023';",start_date)
        self.driver.implicitly_wait(10);
        end_date=self.driver.find_element(By.ID,"txt_PeriodEndDt")
        end_date.clear()
        self.driver.execute_script("arguments[0].value = '23-Jan-2025';",end_date)
        self.driver.execute_script("window.scrollTo(0, 500)")
    

In [9]:
class VillageHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def handle_village(self):
        self.driver.find_element(By.XPATH,"//select[@id='cmb_Village']").click()
        WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='954']"))
        )
        self.driver.find_element(By.CSS_SELECTOR,"option[value='954']").click()
        self.driver.find_element(By.XPATH,"//select[@id='cmb_Village']").click()

In [10]:
class SurveyHandler:
    def __init__(self, driver):
        self.driver = driver
        
    def handle_survey(self):
        self.driver.find_element(By.CSS_SELECTOR,"#txt_SurveyNo").click()
        survery_number=self.driver.find_element(By.CSS_SELECTOR,"#txt_SurveyNo")
        self.driver.execute_script("arguments[0].value = '1';",survery_number)

In [11]:
class AddRecords:
    def __init__(self, driver):
        self.driver = driver
        
    def record_addition(self):
        self.driver.find_element(By.CSS_SELECTOR,"#btn_AddSurvey").click()
        self.driver.implicitly_wait(50);
        

In [12]:
class CaptchaProcessor:
    def __init__(self, driver):
        self.driver = driver
        self.retry_limit = 5

    def captcha_screenshot(self):
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, "captcha")))
        captcha_element = self.driver.find_element(By.ID, "captcha")
        time.sleep(2)
        return captcha_element.screenshot_as_png

    def process_captcha_image(self):
        try:
            captcha_image = self.captcha_screenshot()
            image = Image.open(BytesIO(captcha_image))
            image_np = np.array(image)
            
            gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
            cv2.imwrite("/home/keshu/Documents/CertiSure/step1_grayscale.png", gray)
     
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            cv2.imwrite("/home/keshu/Documents/CertiSure/step2_blurred.png", blurred)
     
            _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            cv2.imwrite("/home/keshu/Documents/CertiSure/step3_binary.png", binary)
            
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
            morphed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
            cv2.imwrite("/home/keshu/Documents/CertiSure/preprocessed_captcha.png", morphed)
            return morphed
        
        except Exception as e:
            print(f"Error processing CAPTCHA image: {e}")
            return None

    def extract_captcha(self):
        try:
            processed_image = self.process_captcha_image()
            if processed_image is None:
                return ""
            
            # Perform OCR
            print("Performing OCR...")
            captcha_text = image_to_string(
                processed_image, 
                config="--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
            )
            return captcha_text.strip()
        except Exception as e:
            print(f"Error extracting CAPTCHA text: {e}")
            return ""

    def enter_captcha(self):
        try:
            # Retry locating CAPTCHA input and search button to handle stale elements
            for _ in range(3):  # Retry up to 3 times
                try:
                    captcha_input = WebDriverWait(self.driver, 5).until(
                        EC.presence_of_element_located((By.ID, "txt_Captcha"))
                    )
                    captcha_text = self.extract_captcha()
                    print(f"Extracted CAPTCHA text: {captcha_text}")
                    
                    time.sleep(3)
                    captcha_input.send_keys(captcha_text)

                    search_button = WebDriverWait(self.driver, 5).until(
                        EC.element_to_be_clickable((By.ID, "btn_SearchDoc"))
                    )
                    search_button.click()
                    time.sleep(2)
                    
                    return
                except StaleElementReferenceException:
                    print("Stale element detected. Retrying CAPTCHA handling...")
                    continue
                except TimeoutException:
                    print("Timeout while locating elements. Retrying...")
                    continue

            print("Failed to handle CAPTCHA after multiple retries.")
        except Exception as e:
            print(f"Error entering CAPTCHA, retrying: {e}")
            
    def handle_captcha_failure(self):
        while True:
            try:
                error_text = WebDriverWait(self.driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#incCaptcha"))
                )
                if error_text:
                    print("CAPTCHA failed, retrying...")
                    self.enter_captcha()
                else:
                    print("CAPTCHA resolved successfully.")
                    break
            except Exception as e:
                print(f"Unexpected error during CAPTCHA handling: {e}")
                break


In [13]:
class PdfDownloader:
    def __init__(self, driver):
        self.driver = driver
        
    def download_pdf(self):
        text=WebDriverWait(self.driver,2).until(
            EC.presence_of_element_located((By.XPATH,"//h2[normalize-space()='Acknowledgement']"))
        )
        if text:
            print("acknowledgement found")
            self.driver.find_element(By.CSS_SELECTOR,"div[class='content col-center'] div h2 a span").click()
            self.driver.implicitly_wait(2)
            self.driver.find_element(By.ID,"okBtn").click()
            search_title=WebDriverWait(self.driver,2).until(
            EC.presence_of_element_located((By.XPATH,"//h2[normalize-space()='Search Encumbrance Certificate']"))
            )
            if search_title:
                field_populater=PopulateField(self.driver)
                field_populater.populate_download() 

In [14]:
class PopulateField:
    def __init__(self,driver):
        self.driver = driver
        
    def populate_download(self):
        zone_handle=ZoneHandler(self.driver)
        zone_handle.select_zone()
        district_handle=DistrictHandler(self.driver)
        district_handle.select_district()
        registrar_handle=RegistrarHandler(self.driver)
        registrar_handle.select_registrar()
        handle_date=DateHandler(self.driver)
        handle_date.populate_dates()
        village_handle=VillageHandler(self.driver)
        village_handle.handle_village()
        field_handle=SurveyHandler(self.driver)
        field_handle.handle_survey()
        record_adder=AddRecords(self.driver)
        record_adder.record_addition()
        captcha_handler=CaptchaProcessor(self.driver)
        captcha_handler.enter_captcha()
        captcha_handler.handle_captcha_failure()
        pdf_downloader=PdfDownloader(self.driver)
        pdf_downloader.download_pdf()

In [15]:
class EncumbranceCertificateDownloader:
    def __init__(self, driver_manager):
        load_dotenv('/root/Desktop/Encumbrance_Automate/cred.env')
        self.url = os.environ.get('URL')
        self.driver = driver_manager.get_driver()
    
    def initialize_driver(self):
        self.driver.get("https://tnreginet.gov.in/")

    def extract_data(self):
        self.initialize_driver()
        site_handler =  SiteNavigator(self.driver)
        site_handler.search_records()
        field_populater=PopulateField(self.driver)
        field_populater.populate_download()
    
    def close_driver(self):
        self.driver.quit()

In [16]:

if __name__ == "__main__":
    driver_manager = DriverManager()
    certificate_extractor = EncumbranceCertificateDownloader(driver_manager)
    certificate_extractor.extract_data()
    #certificate_extractor.close_driver()

Page translated successfully.
Performing OCR...
Extracted CAPTCHA text: EASY
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: NAIC
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: G
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: YRBZN
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: POZD
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: WESAT
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: SEB
CAPTCHA failed, retrying...
Performing OCR...
Extracted CAPTCHA text: EMPL
CAPTCHA failed, retrying...
Error processing CAPTCHA image: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=132.0.2957.127)
Stacktrace:
#0 0x5f06e436a71e <unknown>
#1 0x5f06e3e0aab6 <unknown>
#2 0x5f06e3de436b <unknown>
#3 0x5f06e3e866ed <unknown>
#4 0x5f06e3e9ad9b <unknown>
#5 0x5f06e3e7d503 <unknown>
#6 0x5f06e3e4bf3e <u


KeyboardInterrupt

